In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

In [2]:
import os 
from glob import glob
import random
import numpy as np

In [3]:
os.listdir("cropped_dataset/train")

['1', '2', '.ipynb_checkpoints', '0']

In [4]:
train_dir = "cropped_dataset/train" 
test_dir = "cropped_dataset/test"

# fit 데이터셋 만들기

In [5]:
train_path = glob(train_dir+"/*/*.jpg")
test_path = glob(test_dir+"/*/*.jpg")

In [6]:

random.seed(0)
random.shuffle(train_path)
random.shuffle(test_path)
print(train_path[0])
print(test_path[0])

cropped_dataset/train/1/aug_3_0_4612.jpg
cropped_dataset/test/0/aug_1_2_0_3803.jpg


In [7]:
def get_label(filepath):
    return filepath.split("/")[-2]

In [8]:
train_x = []
for file in train_path:
    gfile = tf.io.read_file(file)
    image = tf.io.decode_image(gfile)
    image = tf.image.resize(image, [50, 50])
    image = tf.image.rgb_to_grayscale(image)
    train_x.append(np.asarray(image))

In [9]:
train_y = [get_label(file) for file in train_path]

In [10]:
test_x = []
for file in test_path:
    gfile = tf.io.read_file(file)
    image = tf.io.decode_image(gfile)
    image = tf.image.resize(image, [50, 50])
    image = tf.image.rgb_to_grayscale(image)
    test_x.append(np.asarray(image))

In [11]:
test_y = [get_label(file) for file in test_path]

In [12]:
train_x[10].shape

(50, 50, 1)

In [13]:
type(train_x[0].shape)
print(type(np.array(train_x[0])))

<class 'numpy.ndarray'>


In [14]:
len(train_y)

13395

In [15]:
np.unique(train_y, return_counts=True)

(array(['0', '1', '2'], dtype='<U1'), array([4478, 4419, 4498]))

In [16]:
np.unique(test_y, return_counts=True)

(array(['0', '1', '2'], dtype='<U1'), array([496, 489, 498]))

In [17]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

In [18]:
train_x = train_x/255
test_x = test_x/255

In [19]:
train_x.shape

(13395, 50, 50, 1)

In [20]:
test_x.shape

(1483, 50, 50, 1)

In [21]:
train_y = train_y.astype("int32")
test_y = test_y.astype("int32")

In [22]:
#train_y = to_categorical(train_y, 3)
#test_y = to_categorical(test_y, 3)

# 모델 정의

In [23]:
input_shape = (50,50,1)
#learning_rate = 0.01
dropout_rate = 0.4
num_classes=3

In [24]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=10
)

In [25]:
inputs = layers.Input(input_shape)
#sequential model에서는 input이 필요 없다
net = layers.Conv2D(16, (3,3), padding="same", activation="relu")(inputs)
net = layers.Conv2D(16, (3,3), padding="same", activation="relu")(net)
net = layers.MaxPool2D((2,2))(net)
net = layers.Conv2D(32, (3,3), padding="same", activation="relu")(net)
net = layers.Conv2D(32, (3,3), padding="same", activation="relu")(net)
net = layers.MaxPool2D((2,2))(net)
net = layers.Conv2D(64, (3,3), padding="same", activation="relu")(net)
net = layers.Conv2D(64, (3,3), padding="same", activation="relu")(net)

net = layers.Flatten()(net)
net = layers.Dense(128)(net)
net = layers.Activation("relu")(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
#주의: binary classification에서는 output dense가 1이다 
net = layers.Activation("softmax")(net)
#net = layers.Activation("sigmoid")(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name="NN")

# Training

In [26]:
EPOCHS = 50

In [27]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        #labels = tf.reshape(labels, [32,1])
        #print(predictions.shape)
        #print(labels.shape)
        
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
    
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [28]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

In [29]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [30]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [31]:
class EarlyStopping:
    def __init__(self):
        self.test_acc_list = []
        self.length = 10
        
    def step(self, acc):
        self.test_acc_list.append(acc.numpy())
        if len(self.test_acc_list) > self.length:
            self.test_acc_list.pop(0)
    def is_stop(self, epoch):
        mean = sum(self.test_acc_list)/len(self.test_acc_list)
        recent_acc = self.test_acc_list[-1]
        if epoch > 10:
            
            if recent_acc - mean <=0.01:
                print("Early Stopping!")
                print("recent_acc: {}, mean acc: {} ".format(recent_acc, mean) )
                return True
            
        return False

In [32]:
es = EarlyStopping()

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
    
    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(template.format(
  
        epoch+1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100
    ))
    
    
    es.step(test_accuracy.result())
    if es.is_stop(epoch):
        break
        
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    
    

Epoch 1, Loss: 0.4200001358985901, Accuracy: 81.11235809326172, Test Loss: 0.2663361132144928, Test Accuracy: 90.49224090576172
Epoch 2, Loss: 0.2149876207113266, Accuracy: 91.43710327148438, Test Loss: 0.20690171420574188, Test Accuracy: 92.11058807373047
Epoch 3, Loss: 0.1289282888174057, Accuracy: 95.28929138183594, Test Loss: 0.1414855420589447, Test Accuracy: 94.67295837402344
Epoch 4, Loss: 0.07976318150758743, Accuracy: 97.22284698486328, Test Loss: 0.08643915504217148, Test Accuracy: 96.42616271972656
Epoch 5, Loss: 0.05394452065229416, Accuracy: 98.08137512207031, Test Loss: 0.13862982392311096, Test Accuracy: 95.549560546875
Epoch 6, Loss: 0.03553951159119606, Accuracy: 98.68607330322266, Test Loss: 0.10439089685678482, Test Accuracy: 96.56102752685547
Epoch 7, Loss: 0.028908010572195053, Accuracy: 99.02202606201172, Test Loss: 0.12389438599348068, Test Accuracy: 96.29130554199219
Epoch 8, Loss: 0.023584693670272827, Accuracy: 99.14893341064453, Test Loss: 0.18175609409809113

# 모델 저장

In [33]:
#model.save("model/", include_optimizer=True, save_format='tf')
model.save('model/fall_detection_basicCNN.h5')